# Analyze Data

In [1]:
import os
import numpy as np
import pygix
import fabio
import tkinter as tk
from tkinter import filedialog
import image_handler as tif
from dezinger import remove_zingers_from_file
from GIWAXS_analyzer import sector_cut
import plotter as plt
import pyqtgraph as pg
import sys
from PIL import Image


if os.name == 'nt':
    win = True
else:
    win = False


default_data_path = os.path.join(os.getcwd(), 'raw_data')
poni = os.path.join(os.getcwd(), 'calibration', 'beamcenter.poni')
raw_data = os.path.join(default_data_path, 'TT5mm-01-benzeneTPP_60min_flip.tif')


"""Inputs"""
sample_orientation = 1       # 1: horizontal up; 2: vertical right; 3: horizontal down; 4: vertical left
incident_angle = 0.12        # angle of incidence of the sample material (usually 0.12 for organics)
tilt_angle = 0              # don't know what this is for to be honest

"""End Inputs"""


%gui qt
print('done')

done


### Find image with GUI and flip it if needed

##### Can run this to use GUI to find file, otherwise, can type it in as 'raw_data' above. If the file is flipped, it will be saved as a new file with the '_flip' appended to it

In [3]:
root = tk.Tk()
if not win:
    root.withdraw()
    root.update()
raw_data = filedialog.askopenfilename(initialdir=default_data_path, title='Select TIFF for analysis',
                                      filetypes=(('TIFFs', '*.tif',), ('all', '*.*')))
root.destroy()
print(f'Using: {raw_data}\n    as calibration image')

"""If in the unflipped folder, we'll need to flip it"""
if f'{os.sep}unflipped{os.sep}' in raw_data.replace('/', os.sep):
    print('Flipping the image for pyFAI-calib2')
    # takes tif at 'raw_data' and rotates 180degrees and then saves in the folder 'save_path'
    # returns new raw_data of the flipped image
    raw_data = tif.flip(filename=raw_data, save_path=default_data_path)

print(f'Flipped image location is\n    {raw_data}')
r, c = tif.size(raw_data)
print(f'The calibration image is {r} x {c} pixels')
# tif.view_tif(raw_data)


Using: /home/etortoric/Documents/GitHub/GIWAXS/raw_data/TT5mm-01-benzeneTPP_60min_flip.tif
    as calibration image
Flipped image location is
    /home/etortoric/Documents/GitHub/GIWAXS/raw_data/TT5mm-01-benzeneTPP_60min_flip.tif
The calibration image is 1102 x 3070 pixels


### Find Poni file (for calibration) with GUI if needed

##### Otherwise you can just define it manually in the first cell.

In [4]:
"""Select file using GUI prompt (can uncomment poni above and skip)"""
root = tk.Tk()
if not win:
    root.withdraw()
    root.update()
poni = filedialog.askopenfilename(initialdir=os.path.join(os.getcwd(), 'calibration'), title='Select poni file',
                                  filetypes=(('PONI', '*.poni',), ('all', '*.*')))
root.destroy()
print(f'Using: {poni}\n    to calibrate analysis')

Using: /home/etortoric/Documents/GitHub/GIWAXS/calibration/agbh3.poni
    to calibrate analysis


### Dezingering

##### Optional step to remove zingers from data file. Will make a new file with '_dz' appended to the name

In [16]:
dezinger_steps = 3
dezing_cutoff = 1.6
dezing_smooth = 3
show = True

if dezinger_steps:
    # if show:
    #     app = pg.QtGui.QApplication(sys.argv)
    raw_data = remove_zingers_from_file(raw_data, dezing_cutoff, dezing_smooth, dezinger_steps, show=show)
    # sys.exit(app.exec_())
    # app.exec_()
print('done')


Dezingering attempt 1
average value of data is 11.156854874465733, but 10.166547053920322 after smoothing.
3383140 pixels in data
Found 34719 zingers

Dezingering attempt 2
average value of data is 11.111272072689868, but 10.125769255780133 after smoothing.
3383140 pixels in data
Found 26670 zingers

Dezingering attempt 3
average value of data is 11.092966297581537, but 10.112253113971045 after smoothing.
3383140 pixels in data
Found 15461 zingers
done


### Load calibration and data with pygix

In [2]:
cal = pygix.Transform()
cal.load(poni)

print(f'Calibrating with:\n    {poni}')

cal.sample_orientation = sample_orientation
cal.incident_angle = incident_angle
cal.tilt_angle = tilt_angle

if sample_orientation == 1:
    orientation_str = 'horizontal'
    direction_str = 'bottom-to-top'
elif sample_orientation == 2:
    orientation_str = 'vertical'
    direction_str = 'left-to-right'
elif sample_orientation == 3:
    orientation_str = 'horizontal'
    direction_str = 'top-to-bottom'
elif sample_orientation == 4:
    orientation_str = 'vertical'
    direction_str = 'right-to-left'
print(f'\nWith a {orientation_str} sample; positive q_z: {direction_str},')
print(f'and an incident angle of {incident_angle}\u00B0')

print(f'\nX-Ray wavelength is {cal._wavelength * 1e10} A')
print(f'\nDetector-sample distance is {cal._dist * 1e3} mm')
print(f'\nDetector pixel size is {cal.detector._pixel1 * 1e6} x {cal.detector._pixel2 * 1e6} \u03BCm')
print(f'\nBeam center is ({cal._poni2 / cal.detector._pixel1}, {cal._poni1 / cal.detector._pixel2}) pixels')
print(f'\nPoint of normal incidence is ({cal._poni1 * 1e3}, {cal._poni2 * 1e3}) mm')
print(f'\nTilt angle is {cal._tilt_angle}\u00B0\n')

print(f'\nLoading data from:\n    {raw_data}')
data = fabio.open(raw_data).data
filename = raw_data.replace('/', os.sep).split(os.sep)[-1].strip('.tif')     # just the part of the string containing the name of the file

data_processed, qxy, qz = cal.transform_reciprocal(data, correctSolidAngle=True, method="bbox")
data_processed = np.fliplr(data_processed)
Image.fromarray(data_processed).save(os.path.join(os.getcwd(), 'processed_data', filename + '.tif'))
tif.view(os.path.join(os.getcwd(), 'processed_data', filename + '.tif'))

Calibrating with:
    /home/etortoric/Documents/GitHub/GIWAXS/calibration/beamcenter.poni

With a horizontal sample; positive q_z: bottom-to-top,
and an incident angle of 0.12°

X-Ray wavelength is 1.54185 A

Detector-sample distance is 163.77355935888423 mm

Detector pixel size is 75.0 x 75.0 μm

Beam center is (1572.2167481699282, 279.9420580059935) pixels

Point of normal incidence is (20.99565435044951, 117.9162561127446) mm

Tilt angle is 0°


Loading data from:
    /home/etortoric/Documents/GitHub/GIWAXS/raw_data/TT5mm-01-benzeneTPP_60min_flip.tif


### Plot In Plane and Out of Plane line cuts

In [ ]:
resolution = 1000  # points
radial_range = (0, 2.5)

q, i_ip = sector_cut((cal, data),                # tuple containing pygix calibratoin and numpy array representing tif image
                     resolution=resolution,      # how many points to use in 1d array output
                     chi_center=78,              # azimuthal center in degrees (0 points up and positive is clockwise)
                     chi_width=10,               # total angle to integrate centered at chi_pos
                     radial_range=radial_range)  # how far to look in the q direction
                     

_, i_oop = sector_cut((cal, data), resolution=resolution, chi_center=0, chi_width=30, radial_range=radial_range)

plt.show_plot2(x=q, y1=i_ip, y2=i_oop, title='In Plane and Out of Plane', y_labels=('In Plane', 'Out of Plane'),
               labels=('q', 'Intensity'), units=('a_inv', 'counts'), colors=('g', 'y'), fontsize=12)
# plot = plt.Plot4('In Plane and Out of Plane', fontsize=12, color='w')
# plt.plot(x=q, y=i_ip, title='In Plane', labels=('q', 'Intensity'), units=('a_inv', 'counts'))
# p1.plot(2, x=q, y=i_oop, title='Out of Plane', labels=('q', 'Intensity'), units=('a_inv', 'counts'))
# pg.plot(3, x=q, y=np.log10(i_ip), title='Log In Plane', labels=('q', 'Log(Intensity)'), units=('a_inv', ''))
# pg.plot(4, x=q, y=np.log10(i_oop), title='Log Out of Plane', labels=('q', 'Log(Intensity)'), units=('a_inv', ''))
print('done')

done


### Find Peaks for Pole Figures

In [3]:
resolution = 1000  # points
radial_range = (0, 2.5)

q_r, i_r = sector_cut((cal, data), resolution = resolution, chi_center=0, chi_width=180, radial_range=radial_range)
q_p, i_p = sector_cut((cal, data_processed), resolution = resolution, chi_center=0, chi_width=180, radial_range=radial_range)
# print(repr(q))
# print(repr(i))
plt.show_plot(x=q_r, y=i_r, title='Peaks Raw', labels=('q', 'Intensity'), units=('a_inv', 'counts'))
plt.show_plot(x=q_p, y=i_p, title='Peaks Processed', labels=('q', 'Intensity'), units=('a_inv', 'counts'))
print('done')

done


In [14]:
data_out = np.concatenate((np.array([q_r]).T, np.array([i_r]).T), axis=1)
print(data_out)
import csv
with open('sector_cut_180.csv','w',newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='#')
    for ii in range(len(q_r)):
        writer.writerow(data_out[ii])
        

[[1.25000015e-03 3.41607928e-02]
 [3.75000045e-03 2.20602319e-01]
 [6.25000075e-03 2.73193240e-01]
 ...
 [2.49375030e+00 3.72230268e+00]
 [2.49625030e+00 3.62202716e+00]
 [2.49875030e+00 3.61597919e+00]]


In [22]:
peak100 = [.600, .650]
peak101 = [.860, .920]
peak001 = [1.22, 1.31]



In [14]:
cake_int, q_cake, chi_cake = cal.transform_polar(data, npt=(1000, 500))

print(cake_int.shape)
print(cake_int[0,:].shape)
print(q_cake.shape)
print(chi_cake.shape)

pg.image(np.rot90(np.rot90(cake_int.T)))

(500, 1000)
(1000,)
(1000,)
(500,)


In [30]:
np.where(np.logical_and(q_cake > peak100[0], q_cake < peak100[1]))

(array([65, 66, 67, 68, 69, 70]),)